In [1]:
!pip install librosa jiwer pydub datasets transformers huggingface_hub
# !pip install datasets
# !pip install datasets transformers==4.28.0
# !pip install torchaudio
# !pip install librosa
# !pip install jiwer
# !pip install huggingface_hub

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00


In [2]:
# prompt: Download data from kaggle

!pip install kaggle

# Upload your kaggle.json file
from google.colab import files
files.upload()

# Make a directory named kaggle and copy kaggle.json file there
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from kaggle
!kaggle datasets download -d vedant2022/common-voice-dataset-version-4


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/vedant2022/common-voice-dataset-version-4
License(s): CC0-1.0
100% 23.2G/23.2G [05:35<00:00, 196MB/s]
100% 23.2G/23.2G [05:35<00:00, 74.2MB/s]


In [3]:
# Unzip the downloaded data
!unzip /content/common-voice-dataset-version-4.zip -d common-voice-dataset


Streaming output truncated to the last 5000 lines.
  inflating: common-voice-dataset/new-clip/common_voice_en_92023.mp3  
  inflating: common-voice-dataset/new-clip/common_voice_en_92024.mp3  
  inflating: common-voice-dataset/new-clip/common_voice_en_92025.mp3  
  inflating: common-voice-dataset/new-clip/common_voice_en_92026.mp3  
  inflating: common-voice-dataset/new-clip/common_voice_en_92027.mp3  
  inflating: common-voice-dataset/new-clip/common_voice_en_92028.mp3  
  inflating: common-voice-dataset/new-clip/common_voice_en_92029.mp3  
  inflating: common-voice-dataset/new-clip/common_voice_en_9203.mp3  
  inflating: common-voice-dataset/new-clip/common_voice_en_92030.mp3  
  inflating: common-voice-dataset/new-clip/common_voice_en_92031.mp3  
  inflating: common-voice-dataset/new-clip/common_voice_en_92032.mp3  
  inflating: common-voice-dataset/new-clip/common_voice_en_92033.mp3  
  inflating: common-voice-dataset/new-clip/common_voice_en_92034.mp3  
  inflating: common-voice-d

In [4]:
from huggingface_hub import notebook_login

notebook_login()


In [5]:
repo_name = "Messam174/wav2vec2-large-xls-r-300m-en"

In [6]:
import pandas as pd
import os
from datasets import Dataset, DatasetDict, Audio

# Define file paths
data_dir = '/content/common-voice-dataset/data-file/'
audio_dir = '/content/common-voice-dataset/new-clip/'

# Load TSV files
train_df = pd.read_csv(os.path.join(data_dir, 'train.tsv'), sep='\t')
test_df = pd.read_csv(os.path.join(data_dir, 'test.tsv'), sep='\t')
validated_df = pd.read_csv(os.path.join(data_dir, 'dev.tsv'), sep='\t')

# Function to map the audio file paths to full paths
def path_to_audio(row):
    return os.path.join(audio_dir, row['path'])

# Add the full audio path
train_df['audio'] = train_df.apply(path_to_audio, axis=1)
test_df['audio'] = test_df.apply(path_to_audio, axis=1)
validated_df['audio'] = validated_df.apply(path_to_audio, axis=1)

# Convert DataFrames to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
valid_dataset = Dataset.from_pandas(validated_df)

# Filter function to keep only specific accents
def filter_accents(example):
    return example['accent'] in ['england', 'us', 'australia']

# # Apply filtering to the datasets
train_dataset_filtered = train_dataset.filter(filter_accents)
test_dataset_filtered = test_dataset.filter(filter_accents)
valid_dataset_filtered = valid_dataset.filter(filter_accents)

# Select the first 5000 examples from the training dataset
train_dataset_filtered = train_dataset_filtered.select(range(10000))


# Cast audio column with resampling to 16000 Hz
train_dataset_filtered = train_dataset_filtered.cast_column('audio', Audio(sampling_rate=16000))
test_dataset_filtered = test_dataset_filtered.cast_column('audio', Audio(sampling_rate=16000))
valid_dataset_filtered = valid_dataset_filtered.cast_column('audio', Audio(sampling_rate=16000))

# Create a DatasetDict object with the filtered data
dataset = DatasetDict({
    'train': train_dataset_filtered,
    'test': test_dataset_filtered,
    'validation': valid_dataset_filtered
})
print(dataset)
# Save the filtered Hugging Face dataset to disk
# common_voice_filtered_dataset.save_to_disk('huggingface_common_voice_filtered_dataset')


Filter:   0%|          | 0/232975 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15531 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15531 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'audio'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'audio'],
        num_rows: 850
    })
    validation: Dataset({
        features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'audio'],
        num_rows: 1437
    })
})


In [7]:
dataset['test'][600]

{'client_id': '52b3b866329974885c7210eed1c1c5a6dfc3a343783e166f1221025a4429ea7c77a5abb10a09b017df04f4c690dc9ff5b0d3e8cdc5f52d7f5809e138f30dc9ca',
 'path': 'common_voice_en_19679780.mp3',
 'sentence': 'In baseball, there are intense rivalries between the four southern schools.',
 'up_votes': 2,
 'down_votes': 1,
 'age': 'thirties',
 'gender': 'male',
 'accent': 'england',
 'audio': {'path': '/content/common-voice-dataset/new-clip/common_voice_en_19679780.mp3',
  'array': array([-6.36646291e-12,  1.18234311e-11, -7.27595761e-12, ...,
          1.78076220e-09,  8.19078139e-10, -4.21948698e-10]),
  'sampling_rate': 16000}}

In [8]:
import os
from datasets import DatasetDict

# Define audio directory
audio_dir = '/content/common-voice-dataset/new-clip/'

# Function to update paths to full paths
def update_path_to_full(dataset):
    dataset = dataset.map(lambda x: {'path': os.path.join(audio_dir, x['path'])})
    return dataset

# Assuming 'dataset' is your DatasetDict
dataset['train'] = update_path_to_full(dataset['train'])
dataset['test'] = update_path_to_full(dataset['test'])
dataset['validation'] = update_path_to_full(dataset['validation'])

# Check the updated dataset structure
print(dataset)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/850 [00:00<?, ? examples/s]

Map:   0%|          | 0/1437 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'audio'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'audio'],
        num_rows: 850
    })
    validation: Dataset({
        features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'audio'],
        num_rows: 1437
    })
})


In [9]:
dataset["train"][400]

{'client_id': '95aa3a613410d0358cc7f9d0ffc328443b3111661313cb0ef92f7da97132621219497665988d524bdf6d6c1d08d2664ace7b29397509a1826e43b82da7240c08',
 'path': '/content/common-voice-dataset/new-clip/common_voice_en_19606749.mp3',
 'sentence': 'Aircraft deployed in Palestine used two wing radiators, to cope with the warmer climate.',
 'up_votes': 2,
 'down_votes': 0,
 'age': 'twenties',
 'gender': 'male',
 'accent': 'us',
 'audio': {'path': '/content/common-voice-dataset/new-clip/common_voice_en_19606749.mp3',
  'array': array([ 0.00000000e+00,  3.63797881e-12, -6.82121026e-12, ...,
          5.10390737e-06,  4.61367472e-06, -2.81211669e-06]),
  'sampling_rate': 16000}}

In [10]:
dataset = dataset.remove_columns(['client_id','up_votes', 'down_votes', 'age', 'gender', 'accent'])
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['path', 'sentence', 'audio'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['path', 'sentence', 'audio'],
        num_rows: 850
    })
    validation: Dataset({
        features: ['path', 'sentence', 'audio'],
        num_rows: 1437
    })
})


In [11]:
dataset["train"][1000]

{'path': '/content/common-voice-dataset/new-clip/common_voice_en_925.mp3',
 'sentence': "You're the biggest person in town!",
 'audio': {'path': '/content/common-voice-dataset/new-clip/common_voice_en_925.mp3',
  'array': array([ 4.54747351e-13, -6.82121026e-13, -1.70530257e-12, ...,
         -1.72842920e-06,  1.48460276e-06,  3.37148504e-06]),
  'sampling_rate': 16000}}

In [12]:
common_voice_train = dataset["train"]
common_voice_test = dataset["test"]
common_voice_val = dataset["validation"]

In [13]:
common_voice_train[1800]

{'path': '/content/common-voice-dataset/new-clip/common_voice_en_19552418.mp3',
 'sentence': 'A Goalkeeper, Swindin was born in Campsall, Doncaster, Yorkshire.',
 'audio': {'path': '/content/common-voice-dataset/new-clip/common_voice_en_19552418.mp3',
  'array': array([-2.03726813e-10, -1.60071068e-10, -5.82076609e-11, ...,
          2.60257220e-05, -3.40803454e-05,  1.90900391e-05]),
  'sampling_rate': 16000}}

In [14]:
common_voice_test[50]

{'path': '/content/common-voice-dataset/new-clip/common_voice_en_17840469.mp3',
 'sentence': 'Those people are kick-ass samba dancers!',
 'audio': {'path': '/content/common-voice-dataset/new-clip/common_voice_en_17840469.mp3',
  'array': array([ 1.81898940e-11,  1.63709046e-11,  2.27373675e-11, ...,
         -1.29400796e-05,  3.09362076e-06,  8.55201506e-06]),
  'sampling_rate': 16000}}

In [15]:
# common_voice_train = common_voice_train.select(range(10000))
# common_voice_test = common_voice_test.select(range(4000))
# common_voice_val = common_voice_val.select(range(4000))
common_voice_train.shape

(10000, 3)

In [16]:
common_voice_train[1000]

{'path': '/content/common-voice-dataset/new-clip/common_voice_en_925.mp3',
 'sentence': "You're the biggest person in town!",
 'audio': {'path': '/content/common-voice-dataset/new-clip/common_voice_en_925.mp3',
  'array': array([ 4.54747351e-13, -6.82121026e-13, -1.70530257e-12, ...,
         -1.72842920e-06,  1.48460276e-06,  3.37148504e-06]),
  'sampling_rate': 16000}}

In [17]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(common_voice_train.remove_columns(["path", "audio"]))

,sentence
0,He succumbed to drugs and was a dedicated heroin addicted for ten years.
1,She avoids artificial fibres such as polyester or nylon.
2,Baldwin made a number of switchers with similar dimensions and body styles.
3,Clare were the defending champions but they did not enter the championship.
4,Several firefighters had to undergo treatment for respiratory irritation.
5,Trains draw power from the electric third rail.
6,"However, his wife and daughters did not survive its liquidation."
7,Give me the list of animated movies playing at the closest movie house
8,"The Anamudi peak area is also habitat of a unique frog ""Raorchestes resplendens""."
9,A man and a dog sit on a bench near a body of water


In [18]:
import re

# Define the characters to ignore
chars_to_ignore_regex = r'[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

# Function to remove special characters
def remove_special_characters(batch):
    # Ensure sentence is a string before applying regex
    if isinstance(batch["sentence"], str):
        batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    return batch

# Apply the function to remove special characters
# dataset = dataset.map(remove_special_characters)
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)
common_voice_val = common_voice_val.map(remove_special_characters)

# print(dataset)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/850 [00:00<?, ? examples/s]

Map:   0%|          | 0/1437 [00:00<?, ? examples/s]

In [19]:
show_random_elements(common_voice_train .remove_columns(["path", "audio"]))


,sentence
0,he is carrying important confidential papers for a meeting
1,galamian taught concurrently at curtis juilliard and meadowmount schools
2,hes had notable performances in several heavy metal and hard rock groups
3,a bridge crosses a ravine high up amongst the mountains
4,a small child hangs upside down from the back of an adult
5,sunni islam also has the idea of beatific vision
6,rodney griffin was the first official groundskeeper of the facility
7,trains draw power from the electric third rail
8,all listings are with original names
9,he received several awards from the beard liberation front including beard of the year


In [20]:
import re

# Modify the extract_all_chars function to filter out non-standard characters
def extract_all_chars(batch):
    # Remove None values and replace them with empty strings
    sentences = [s if s is not None else "" for s in batch["sentence"]]
    all_text = " ".join(sentences)
    # Keep only standard alphabet characters and basic punctuation
    all_text = re.sub(r'[^a-zA-Z0-9 .,!?\'"]', '', all_text)
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

# Apply the function using map
# vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])

vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)
vocab_val = common_voice_val.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_val.column_names)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/850 [00:00<?, ? examples/s]

Map:   0%|          | 0/1437 [00:00<?, ? examples/s]

In [21]:
# vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]) | set(vocabs["validation"]["vocab"][0]))
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]) | set(vocab_val["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict


{'b': 0,
 'm': 1,
 'i': 2,
 's': 3,
 'e': 4,
 'u': 5,
 'v': 6,
 't': 7,
 'x': 8,
 'f': 9,
 ' ': 10,
 'p': 11,
 'g': 12,
 'h': 13,
 'l': 14,
 'o': 15,
 'y': 16,
 'a': 17,
 'w': 18,
 'r': 19,
 'c': 20,
 'k': 21,
 'n': 22,
 'q': 23,
 'j': 24,
 'z': 25,
 'd': 26}

In [22]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]


In [23]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))


29


In [24]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


In [25]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [26]:
tokenizer.push_to_hub(repo_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Messam174/wav2vec2-large-xls-r-300m-en/commit/6f1b3eb6b505eae5df12970bb26a9910d88f912b', commit_message='Upload tokenizer', commit_description='', oid='6f1b3eb6b505eae5df12970bb26a9910d88f912b', pr_url=None, pr_revision=None, pr_num=None)

## Create Wav2Vec2 Feature Extractor


In [27]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)


In [28]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [29]:
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='', vocab_size=29, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	27: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	28: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	29: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	30: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False

## Preprocess Data


In [30]:
common_voice_train[7]

{'path': '/content/common-voice-dataset/new-clip/common_voice_en_19687172.mp3',
 'sentence': 'despite running as a joke candidates have won the executive race in the past',
 'audio': {'path': '/content/common-voice-dataset/new-clip/common_voice_en_19687172.mp3',
  'array': array([-1.81898940e-12,  9.54969437e-12,  0.00000000e+00, ...,
          6.25912605e-07, -1.28090107e-07,  1.54477561e-06]),
  'sampling_rate': 16000}}

In [31]:
from datasets import Audio


In [32]:

# Cast the 'audio' column to the correct Audio format with 16 kHz sampling rate
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16000))
common_voice_val = common_voice_val.cast_column("audio", Audio(sampling_rate=16000))

# If you have a validation set
# common_voice_valid = common_voice_valid.cast_column("audio", Audio(sampling_rate=16000))

print("Audio casting completed.")


Audio casting completed.


In [33]:
common_voice_train[2000]


{'path': '/content/common-voice-dataset/new-clip/common_voice_en_19753927.mp3',
 'sentence': 'some  in the above  live in great comfort',
 'audio': {'path': '/content/common-voice-dataset/new-clip/common_voice_en_19753927.mp3',
  'array': array([-2.18278728e-11,  5.45696821e-11,  3.63797881e-11, ...,
          1.41000510e-06, -3.82759481e-07, -1.80983693e-06]),
  'sampling_rate': 16000}}

In [34]:
common_voice_train[1000]

{'path': '/content/common-voice-dataset/new-clip/common_voice_en_925.mp3',
 'sentence': 'youre the biggest person in town',
 'audio': {'path': '/content/common-voice-dataset/new-clip/common_voice_en_925.mp3',
  'array': array([ 4.54747351e-13, -6.82121026e-13, -1.70530257e-12, ...,
         -1.72842920e-06,  1.48460276e-06,  3.37148504e-06]),
  'sampling_rate': 16000}}

In [35]:
import IPython.display as ipd
import numpy as np
import random

# rand_int = random.randint(0, len(dataset["train"])-1)

# print(dataset["train"][rand_int]["sentence"])
# ipd.Audio(data=np.asarray(dataset["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

rand_int = random.randint(0, len(common_voice_train))

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

over the past several years denison university has increasingly attracted a diverse student population


In [36]:
# # Access a random sample from the training set
# rand_int = random.randint(0, len(dataset["train"])-1)

# # Access the example
# example = dataset["train"][rand_int]

# # Display the details
# print("Target text:", example["sentence"])
# print("Input array shape:", np.asarray(example["audio"]["array"]).shape)
# print("Sampling rate:", example["audio"]["sampling_rate"])

rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])


Target text: it was also the first album since how could hell be any worse
Input array shape: (82560,)
Sampling rate: 16000


In [37]:
# audio = common_voice_train[998]["audio"]
# print(processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0])


In [38]:
def prepare_dataset(batch):
    try:
        # Ensure batch has the required columns
        if "audio" not in batch or "sentence" not in batch:
            raise KeyError("Missing 'audio' or 'sentence' column")

        # Extract audio data
        audio = batch["audio"]

        # Process the audio (this returns a list of input values for Wav2Vec2)
        input_values = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]

        # Length of the input values
        batch["input_values"] = input_values
        batch["input_length"] = len(input_values)

        # Process the sentence into labels (IDs)
        with processor.as_target_processor():
            batch["labels"] = processor(batch["sentence"]).input_ids

    except FileNotFoundError as e:
        print(f"File not found: {e}")
        # Mark the batch as failed for this file (could set input_values as None or empty)
        batch["input_values"] = None
        batch["input_length"] = 0
        batch["labels"] = None
    except KeyError as e:
        print(f"KeyError: {e}")
        batch["input_values"] = None
        batch["input_length"] = 0
        batch["labels"] = None
    except Exception as e:
        print(f"General error in processing: {e}")
        batch["input_values"] = None
        batch["input_length"] = 0
        batch["labels"] = None



    return batch


In [39]:
common_voice_train[500]['audio']['path']


'/content/common-voice-dataset/new-clip/common_voice_en_19602352.mp3'

In [41]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_18335527.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_18335528.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_18335529.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_18335530.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_18335531.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_19946725.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_19946726.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_19946727.mp3'
File not found: 

In [42]:
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)


Map:   0%|          | 0/850 [00:00<?, ? examples/s]

File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_17758449.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_17899678.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_255533.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_17636020.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_17636024.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_17274621.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_17274622.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_615156.mp3'
File not found: [Err

In [43]:
common_voice_val = common_voice_val.map(prepare_dataset, remove_columns=common_voice_val.column_names)


Map:   0%|          | 0/1437 [00:00<?, ? examples/s]

File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_17880021.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_20005985.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_20005986.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_20005987.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_20005990.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_19677494.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_19677497.mp3'
File not found: [Errno 2] No such file or directory: '/content/common-voice-dataset/new-clip/common_voice_en_19677502.mp3'
File not found: 

In [44]:
def filter_none_inputs(batch):
    return batch["input_values"] is not None

common_voice_train = common_voice_train.filter(filter_none_inputs)
common_voice_val = common_voice_val.filter(filter_none_inputs)
common_voice_test = common_voice_test.filter(filter_none_inputs)


Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1437 [00:00<?, ? examples/s]

Filter:   0%|          | 0/850 [00:00<?, ? examples/s]

In [45]:
max_input_length_in_sec = 5.0
common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])


Filter:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available


True


In [46]:
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [47]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [48]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00


In [49]:
import evaluate

# Load the Word Error Rate (WER) metric
wer = evaluate.load("wer")

# Example: Predictions and references
predictions = ["hello world", "the cat sat on the mat"]
references = ["hello world", "the cat is on the mat"]

# Compute WER
wer_score = wer.compute(predictions=predictions, references=references)
print(f"WER: {wer_score}")


WER: 0.125


In [50]:
wer_metric = evaluate.load("wer")


In [51]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [52]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
model.freeze_feature_extractor()


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2177: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [54]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  overwrite_output_dir= True,
  group_by_length=True,
  per_device_train_batch_size=10,
  gradient_accumulation_steps=2,
  evaluation_strategy="epoch",
  num_train_epochs=7,
  fp16=True,
  gradient_checkpointing=True,
  save_strategy="epoch",
  logging_steps=132,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  load_best_model_at_end=True,
  push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="epoch",
  num_train_epochs=15,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)


In [55]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [56]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.auto

Epoch,Training Loss,Validation Loss,Wer
0,7.966900,3.063552,1.000000
2,2.962800,1.996155,1.030066
4,1.039800,0.990714,0.693907
6,0.551400,0.828128,0.569801


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.auto

TrainOutput(global_step=1197, training_loss=2.332461212910307, metrics={'train_runtime': 3517.1726, 'train_samples_per_second': 6.813, 'train_steps_per_second': 0.34, 'total_flos': 2.961091308721513e+18, 'train_loss': 2.332461212910307, 'epoch': 6.979591836734694})

In [57]:
trainer.push_to_hub()


CommitInfo(commit_url='https://huggingface.co/Messam174/wav2vec2-large-xls-r-300m-en/commit/3324e312d9749b231ca1a4cca4b250388a819659', commit_message='End of training', commit_description='', oid='3324e312d9749b231ca1a4cca4b250388a819659', pr_url=None, pr_revision=None, pr_num=None)

In [58]:
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'eval_loss': 0.8281283974647522,
 'eval_wer': 0.5698013245033112,
 'eval_runtime': 68.9606,
 'eval_samples_per_second': 12.036,
 'eval_steps_per_second': 1.508,
 'epoch': 6.979591836734694}

In [2]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("Messam174/wav2vec2-large-xls-r-300m-en")
processor = Wav2Vec2Processor.from_pretrained("Messam174/wav2vec2-large-xls-r-300m-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

### Evaluate

In [72]:
val = dataset["validation"]

In [7]:
from transformers import AutoModelForCTC, Wav2Vec2Processor, Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(repo_name)#.to("cuda")
processor = Wav2Vec2Processor.from_pretrained(repo_name)

In [83]:
input_dict = processor(common_voice_val[100]["input_values"],
                       return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [86]:
print(val[100]["sentence"])
ipd.Audio(data=val[100]["audio"]["array"], autoplay=True, rate=16000)

Shoe the movie showings for the closest cinema showing international movies


In [85]:
print("Prediction:")
print(processor.decode(pred_ids))

# print("\nReference:")
# print(common_voice_test[600]["sentence"].lower())

Prediction:
shows a movi showins for the closest cenema showing international rivies


In [1]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/1

In [13]:
!pip install --upgrade gradio  # Upgrade Gradio

In [16]:
import gradio as gr
import librosa
import torch
def transcribe(audio_filepath):
  """
  Transcribes audio using the Wav2Vec2 model.

  Args:
    audio_filepath: The path to the audio file to transcribe.

  Returns:
    The transcribed text.
  """
  try:
    # Load the audio data from the file path using librosa
    audio_data, sr = librosa.load(audio_filepath, sr=16000)

    input_dict = processor(audio_data, sampling_rate=sr, return_tensors="pt", padding=True)
    logits = model(input_dict.input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)[0]
    transcription = processor.decode(pred_ids)
    return transcription
  except Exception as e:
    return f"Error during transcription: {e}"


iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(type="filepath"),
    outputs="text",
    title="Wav2Vec2 Speech Transcription",
    description="Transcribe speech from your microphone using a fine-tuned Wav2Vec2 model.",
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c74a2670952e7f8be3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
